In [1]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU, Convolution1D,Embedding,Convolution2D
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 123
numpy.random.seed(seed)

In [3]:
# load dataset
dataframe = pd.read_csv("C:\\Users\\Sai\\Desktop\\Insofe\\exams\\PHD\\MLP_data.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[1:,0:54].astype(float)
Y = dataset[1:,-1]

In [4]:
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [5]:
encoded_Y.shape
#encoded_Y

(3156,)

In [6]:
X.shape

(3156, 54)

In [7]:
a=numpy.reshape(X, (1, X.shape[0], X.shape[1],1))
a.shape

(1, 3156, 54, 1)

In [8]:
X_train_r = numpy.zeros((len(X), 54, 1))
X_train_r[:, :, 0] = X[:, :54]
X_train_r

array([[[ 6.],
        [ 0.],
        [ 1.],
        ..., 
        [ 0.],
        [ 1.],
        [ 0.]],

       [[ 4.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 1.]],

       [[ 4.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 1.]],

       ..., 
       [[ 6.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 1.]],

       [[ 8.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 1.]],

       [[ 4.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 0.]]])

In [9]:
b=numpy.reshape(Y,(1,3156,1,1))
b.shape

(1, 3156, 1, 1)

In [20]:
def create_baseline():
    # create model
    model = Sequential()
    #model.add(Embedding(54,54,input_length=54))
    #model.add(Conv1D(activation='relu', input_shape=(54,), strides=1, filters=8, kernel_size=8))
    model.add(Convolution1D(nb_filter=128, filter_length=1, input_shape=(54, 1)))
    model.add(Dropout(0.3))
    model.add(Conv1D(activation='relu', strides=1, filters=32, kernel_size=8))
    model.add(Dropout(0.2))
    model.add(Conv1D(activation='relu', strides=1, filters=32, kernel_size=8))
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Flatten())
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [21]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=20, batch_size=128, verbose=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_train_r, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\Users\Sai\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(54, 1), filters=128, kernel_size=1)`
  


Epoch 1/20
2524/2524 [==============================] - 7s 3ms/step - loss: 0.6897 - acc: 0.5333
Epoch 2/20
2524/2524 [==============================] - 2s 891us/step - loss: 0.6296 - acc: 0.6799
Epoch 3/20
2524/2524 [==============================] - 2s 852us/step - loss: 0.5609 - acc: 0.7235
Epoch 4/20
2524/2524 [==============================] - 2s 820us/step - loss: 0.5364 - acc: 0.7342
Epoch 5/20
2524/2524 [==============================] - 2s 757us/step - loss: 0.5175 - acc: 0.7433
Epoch 6/20
2524/2524 [==============================] - 2s 736us/step - loss: 0.4659 - acc: 0.7880
Epoch 7/20
2524/2524 [==============================] - 2s 704us/step - loss: 0.3972 - acc: 0.8372
Epoch 8/20
2524/2524 [==============================] - 2s 705us/step - loss: 0.3712 - acc: 0.8526
Epoch 9/20
2524/2524 [==============================] - 2s 977us/step - loss: 0.3667 - acc: 0.8621
Epoch 10/20
2524/2524 [==============================] - 2s 978us/step - loss: 0.3701 - acc: 0.8590
Epoch 11/20

2526/2526 [==============================] - 12s 5ms/step - loss: 0.6876 - acc: 0.5907
Epoch 2/20
2526/2526 [==============================] - 2s 889us/step - loss: 0.6284 - acc: 0.6536
Epoch 3/20
2526/2526 [==============================] - 2s 839us/step - loss: 0.5636 - acc: 0.7245
Epoch 4/20
2526/2526 [==============================] - 2s 735us/step - loss: 0.5351 - acc: 0.7435
Epoch 5/20
2526/2526 [==============================] - 6s 2ms/step - loss: 0.5082 - acc: 0.7553
Epoch 6/20
2526/2526 [==============================] - 2s 810us/step - loss: 0.4768 - acc: 0.7712
Epoch 7/20
2526/2526 [==============================] - 2s 836us/step - loss: 0.4399 - acc: 0.8009
Epoch 8/20
2526/2526 [==============================] - 7s 3ms/step - loss: 0.3841 - acc: 0.8547
Epoch 9/20
2526/2526 [==============================] - 2s 830us/step - loss: 0.3574 - acc: 0.8638
Epoch 10/20
2526/2526 [==============================] - 7s 3ms/step - loss: 0.3545 - acc: 0.8662
Epoch 11/20
2526/2526 [====

In [28]:
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X_train_r, encoded_Y):
    # create model
    model = Sequential()
    #model.add(Embedding(54,54,input_length=54))
    #model.add(Conv1D(activation='relu', input_shape=(54,), strides=1, filters=8, kernel_size=8))
    model.add(Convolution1D(nb_filter=128, filter_length=1, input_shape=(54, 1)))
    model.add(Dropout(0.3))
    model.add(Conv1D(activation='relu', strides=1, filters=32, kernel_size=8))
    model.add(Dropout(0.2))
    model.add(Conv1D(activation='relu', strides=1, filters=32, kernel_size=8))
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Flatten())
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X_train_r[train], encoded_Y[train], epochs=20, batch_size=128, verbose=1)
    # evaluate the model
    scores = model.evaluate(X_train_r[test], encoded_Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

C:\Users\Sai\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(54, 1), filters=128, kernel_size=1)`
  if __name__ == '__main__':


Epoch 1/20
2524/2524 [==============================] - 10s 4ms/step - loss: 0.6892 - acc: 0.5353
Epoch 2/20
2524/2524 [==============================] - 3s 1ms/step - loss: 0.6304 - acc: 0.6704
Epoch 3/20
2524/2524 [==============================] - 2s 831us/step - loss: 0.5654 - acc: 0.7163
Epoch 4/20
2524/2524 [==============================] - 2s 746us/step - loss: 0.5387 - acc: 0.7342
Epoch 5/20
2524/2524 [==============================] - 2s 720us/step - loss: 0.5077 - acc: 0.7607
Epoch 6/20
2524/2524 [==============================] - 2s 738us/step - loss: 0.4541 - acc: 0.7928
Epoch 7/20
2524/2524 [==============================] - 2s 746us/step - loss: 0.3927 - acc: 0.8510
Epoch 8/20
2524/2524 [==============================] - 2s 759us/step - loss: 0.3738 - acc: 0.8546
Epoch 9/20
2524/2524 [==============================] - 2s 768us/step - loss: 0.3616 - acc: 0.8617
Epoch 10/20
2524/2524 [==============================] - 2s 744us/step - loss: 0.3642 - acc: 0.8629
Epoch 11/20


2526/2526 [==============================] - 2s 862us/step - loss: 0.6299 - acc: 0.6595
Epoch 3/20
2526/2526 [==============================] - 2s 823us/step - loss: 0.5589 - acc: 0.7237
Epoch 4/20
2526/2526 [==============================] - 2s 768us/step - loss: 0.5420 - acc: 0.7296
Epoch 5/20
2526/2526 [==============================] - 2s 855us/step - loss: 0.5113 - acc: 0.7546
Epoch 6/20
2526/2526 [==============================] - 3s 1ms/step - loss: 0.4467 - acc: 0.7949
Epoch 7/20
2526/2526 [==============================] - 2s 931us/step - loss: 0.3831 - acc: 0.8488
Epoch 8/20
2526/2526 [==============================] - 2s 742us/step - loss: 0.3624 - acc: 0.8622
Epoch 9/20
2526/2526 [==============================] - 2s 742us/step - loss: 0.3555 - acc: 0.8670
Epoch 10/20
2526/2526 [==============================] - 4s 2ms/step - loss: 0.3496 - acc: 0.8694
Epoch 11/20
2526/2526 [==============================] - 2s 941us/step - loss: 0.3397 - acc: 0.8721
Epoch 12/20
2526/2526 [

In [32]:
test=pd.read_csv("C:\\Users\\Sai\\Desktop\\Insofe\\exams\\PHD\\MLP_Test.csv", header=None)
data_test = test.values
X_test=X = data_test[1:,:].astype(float)

In [33]:
X_test_r = numpy.zeros((len(X_test), 54, 1))
X_test_r[:, :, 0] = X_test[:, :54]
X_test_r

array([[[ 4.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 1.]],

       [[ 8.],
        [ 0.],
        [ 0.],
        ..., 
        [ 1.],
        [ 0.],
        [ 1.]],

       [[ 8.],
        [ 1.],
        [ 0.],
        ..., 
        [ 1.],
        [ 1.],
        [ 1.]],

       ..., 
       [[ 4.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 1.]],

       [[ 8.],
        [ 0.],
        [ 0.],
        ..., 
        [ 1.],
        [ 0.],
        [ 1.]],

       [[ 4.],
        [ 1.],
        [ 0.],
        ..., 
        [ 0.],
        [ 1.],
        [ 0.]]])

In [34]:
predictions = model.predict(X_test_r)
ans=pd.DataFrame(predictions)

In [35]:
print(ans)

             0
0     0.883777
1     0.079021
2     0.853700
3     0.870237
4     0.970901
5     0.027572
6     0.345894
7     0.972616
8     0.007909
9     0.117080
10    0.807043
11    0.042244
12    0.096034
13    0.707615
14    0.318836
15    0.048015
16    0.932547
17    0.742808
18    0.807043
19    0.819233
20    0.251104
21    0.982022
22    0.963368
23    0.782681
24    0.092526
25    0.130380
26    0.117624
27    0.419846
28    0.930848
29    0.057405
...        ...
1023  0.694201
1024  0.804739
1025  0.073763
1026  0.830191
1027  0.966669
1028  0.065054
1029  0.032242
1030  0.410724
1031  0.877657
1032  0.946433
1033  0.963187
1034  0.069978
1035  0.319257
1036  0.926620
1037  0.038682
1038  0.071513
1039  0.718225
1040  0.456419
1041  0.902880
1042  0.963434
1043  0.793378
1044  0.455768
1045  0.963895
1046  0.830195
1047  0.731982
1048  0.400333
1049  0.718919
1050  0.067447
1051  0.869058
1052  0.055301

[1053 rows x 1 columns]


In [36]:
ans.to_csv('CNN_result.csv', float_format='%.6f', index=False)